In [3]:
import dataloader_test as dlt


train_files = [f'../data/modified_ndws/train_conus_west_ndws_0{i:02}.tfrecord' for i in range(39)]
dataset = dlt.MultiTFRecordDataset(train_files)
loader = dlt.DataLoader(dataset, batch_size=32)


In [2]:
# Setup 1: ignore all problematic variables; that is, give them units that exclude them from the nullspace of the dimensional matrix
from data import drop_prob_setup as setup

dataset = dlt.NondimFireDataset(train_files, setup.units_, positive=setup.positive, constants=setup.constants)
loader = dlt.DataLoader(dataset, batch_size=32)

In [7]:
# Setup 2: include all problematic variables; that is, call them dimensionless to include them in the nullspace of the dimensional matrix
from data import keep_prob_setup as setup

dataset = dlt.NondimFireDataset(train_files, setup.units_, positive=setup.positive, constants=setup.constants)
loader = dlt.DataLoader(dataset, batch_size=32)